In [ ]:
!nvidia-smi

Wed Mar 25 05:44:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import os 
os.chdir('drive/My Drive/')

In [ ]:
import zipfile

f = zipfile.ZipFile("xlnet/chinese_xlnet_base_pytorch.zip",'r')
for file in f.namelist():
  f.extract(file,'xlnet/')
f.close

<bound method ZipFile.close of <zipfile.ZipFile filename='xlnet/chinese_xlnet_base_pytorch.zip' mode='r'>>

In [ ]:
!pip install pytorch_transformers

     |████████████████████████████████| 184kB 20.5MB/s 
     |████████████████████████████████| 870kB 36.3MB/s 
     |████████████████████████████████| 1.0MB 40kB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=65cbd9d38d0af46e386db3ddea628ab131c7926631ee81eda313d1fba9cd52c3
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [ ]:
from pytorch_transformers.modeling_xlnet import XLNetForSequenceClassification, XLNetConfig
import torch
import pandas as pd
import numpy as np
from pytorch_transformers import AdamW, WarmupLinearSchedule
from pytorch_transformers.tokenization_xlnet import XLNetTokenizer
from pytorch_transformers.tokenization_bert import BertTokenizer
from tqdm import tqdm
import torch.utils.data.dataloader as dataloader
from sklearn.metrics import accuracy_score,recall_score,f1_score
# from pytorch_pretrained_bert import BertForSequenceClassification,BertModel
# from pytorch_transformers import Adamw
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.model_selection import StratifiedKFold
import time

In [ ]:
MAX_SEQUENCE_LENGTH = 140
batch_size = 64
epochs = 1500
input_categories = '微博中文内容'
output_categories = '情感倾向'

In [ ]:
train_0 = pd.read_csv("data/train_1")
val_0 = pd.read_csv("data/val_1")

In [ ]:
train_0 = train_0[:128]
val_0 = val_0[:128]

In [ ]:
def _convert_to_transformer_inputs(instance,tokenizer,max_sequence_length):
    def return_id(str1,truncation_strategy,length):
        inputs = tokenizer.tokenize(str1)
        if len(inputs) > 137:
          inputs = inputs[:137]
        inputs = ["<sep>"]+ inputs + ["<sep>"]+["<cls>"]
        input_ids =  tokenizer.convert_tokens_to_ids(inputs)
        # print(len(input_ids))
        input_masks = [1] * len(input_ids)
        # print(len(input_masks))
        input_segments = [1] * (len(inputs)-1)+[2]
        # print(len(input_segments))
        padding_length = length - len(input_ids)
#         padding_id = tokenizer.pad_token_id
        input_ids = ([0] * padding_length) + input_ids 
        input_masks = ([0] * padding_length) + input_masks 
        input_segments = ([4] * padding_length) + input_segments 
        # if len(input_ids) != 200:
        #   print(str1,len(input_ids))
        return [input_ids, input_masks, input_segments]
    
    input_ids,input_masks,input_segments = return_id(instance, 'longest_first', max_sequence_length)
    return [input_ids, input_masks,input_segments]


In [ ]:
def compute_input_arrays(df,columns,tokenizer,max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for instance in tqdm(df[columns]):
        
        ids, masks, segments = \
        _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    # print(input_ids)

    return input_ids, input_masks, input_segments

def compute_output_arrays(df,columns):
    return np.asarray(df[columns].astype(int) + 1)

def data_loader(input_ids,input_masks,input_segments,label_ids):
    all_input_ids = torch.tensor(input_ids, dtype=torch.long)
    all_input_mask = torch.tensor(input_masks, dtype=torch.long)
    all_segment_ids = torch.tensor(input_segments, dtype=torch.long)
    all_label = torch.tensor(label_ids, dtype=torch.long)
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)
    train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=True)
    return train_dataloader

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet/spiece.model")
t_input_ids, t_input_masks, t_input_segments = compute_input_arrays(train_0,input_categories,tokenizer,MAX_SEQUENCE_LENGTH)

t_label_ids = compute_output_arrays(train_0, output_categories)
train_dataloader = data_loader(t_input_ids, t_input_masks, t_input_segments,t_label_ids)
v_input_ids, v_input_masks, v_input_segments = compute_input_arrays(val_0,input_categories,tokenizer,MAX_SEQUENCE_LENGTH)
v_label_ids = compute_output_arrays(val_0, output_categories)
val_dataloader = data_loader(v_input_ids, v_input_masks, v_input_segments,v_label_ids)
device = torch.device("cuda:0")
# bert_model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path='bert-base-chinese', num_labels=3)
xlnet_model = XLNetForSequenceClassification.from_pretrained("xlnet/", num_labels=3)
param_optimizer = list(xlnet_model.named_parameters())
# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer]
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
criterion = nn.CrossEntropyLoss()
xlnet_model.to(device)

100%|██████████| 19982/19982 [00:04<00:00, 4403.83it/s]


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:
import torch.nn as nn
import torch.nn.functional as F 

def train(model, iterator, optimizer, criterion, device):
    start = time.time()
    model.train()
    epoch_loss = 0
    i = 0
    
    for input_ids, segment_ids,input_mask,label_ids in iterator:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        loss = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask, labels=label_ids)
        model.zero_grad()
        # print(loss[0])
        # print("-----")
        # y_pred_label = loss[1].cpu()
        # loss = F.cross_entropy(y_pred, label_ids)
        epoch_loss += loss[0]
        y_pred_c = loss[1].argmax(dim=1).cpu()
        # print(y_pred_c)
        label_ids_c = label_ids.cpu()
        # print(label_ids_c)
        if i %300 == 0:
          # print("i",i,"loss",loss[0].cpu())

          print("i",i,"loss",loss[0].cpu(),"train acc:",accuracy_score(y_pred_c,label_ids_c),"train rec:",recall_score(y_pred_c,label_ids_c,average='macro'),"train f1",f1_score(y_pred_c,label_ids_c,average='macro'))
        loss[0].backward()
        optimizer.step() 
        i += 1
    end = time.time()
    runtime = end-start
    print('time: %.2f' , runtime)
    return epoch_loss / len(iterator)
def deval(model, iterator, criterion, device):
    model.eval()
    abblist = []
    n = 0
    f1 = 0
    acc = 0
    rec = 0
    with torch.no_grad():
      for input_ids,  segment_ids,input_mask,label_ids in iterator:
        
        n += 1
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        output = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask)
        # print(output)
        y_pred_label = output[0].argmax(dim=1).cpu()
        acc += accuracy_score(y_pred_label,label_ids)
        rec += recall_score(y_pred_label,label_ids,average='macro')
        f1 += f1_score(y_pred_label,label_ids,average='macro')
      print("train acc :",acc/n, "rec:",rec/n,"f1:",f1/n,'n:',n)


In [ ]:
stat = 0 
for i in range(epochs):
    train_loss = train(xlnet_model, train_dataloader, optimizer, criterion, device) 
    lss = "xlnet_model/p100_xlnet_"+str(i)+"_.pk1"
    if i == 0 :
      stat = train_loss
    if i != 0 :
      if stat-train_loss < 0.005 :
        break
      if stat-train_loss < 0 :
        break
    torch.save(xlnet_model.state_dict(), lss)
    print("train loss: ", train_loss)
    deval(xlnet_model, val_dataloader, criterion, device)
    # deval(xlnet_model, train_dataloader, criterion, device)
torch.save(xlnet_model.state_dict(), "xlnet/params_bertend.pk1")


i 0 loss tensor(1.0825, grad_fn=<CopyBackwards>) train acc: 0.484375 train rec: 0.34595959595959597 train f1 0.3279778035875597
i 300 loss tensor(0.5132, grad_fn=<CopyBackwards>) train acc: 0.75 train rec: 0.7260452961672473 train f1 0.6866873065015479
i 600 loss tensor(0.6601, grad_fn=<CopyBackwards>) train acc: 0.71875 train rec: 0.6746031746031745 train f1 0.6476327872921321
i 900 loss tensor(0.7097, grad_fn=<CopyBackwards>) train acc: 0.78125 train rec: 0.7420634920634921 train f1 0.7277777777777779
i 1200 loss tensor(0.6036, grad_fn=<CopyBackwards>) train acc: 0.734375 train rec: 0.7641196013289037 train f1 0.6901748040988548
time: %.2f 3074.321467399597
train loss:  tensor(0.6108, device='cuda:0', grad_fn=<DivBackward0>)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train acc : 0.7532376768598813 rec: 0.7223647299998486 f1: 0.7195447134773881 n: 313
i 0 loss tensor(0.5552, grad_fn=<CopyBackwards>) train acc: 0.71875 train rec: 0.6887125220458553 train f1 0.7049247049247049
i 300 loss tensor(0.4544, grad_fn=<CopyBackwards>) train acc: 0.796875 train rec: 0.7195767195767195 train f1 0.7045576146699742
i 600 loss tensor(0.4945, grad_fn=<CopyBackwards>) train acc: 0.8125 train rec: 0.7390572390572391 train f1 0.7323232323232324
i 900 loss tensor(0.5348, grad_fn=<CopyBackwards>) train acc: 0.78125 train rec: 0.7527993109388458 train f1 0.7255689424364123
i 1200 loss tensor(0.5731, grad_fn=<CopyBackwards>) train acc: 0.734375 train rec: 0.7085137085137084 train f1 0.6884952585435679
time: %.2f 3081.0976371765137
train loss:  tensor(0.5269, device='cuda:0', grad_fn=<DivBackward0>)
train acc : 0.7595704016430854 rec: 0.7458146165118041 f1: 0.7140123829379131 n: 313
i 0 loss tensor(0.4402, grad_fn=<CopyBackwards>) train acc: 0.828125 train rec: 0.816683316